<a href="https://colab.research.google.com/github/wanickols01/LearningMLandDL/blob/main/SparkTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's install pyspark
##### Remember to upload subset.small.tsv (in github repo)


In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


Let's import some libraries we'll need 

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF


Create the session and context

In [3]:
# Boilerplate Spark stuff:
conf = SparkConf().setMaster("local").setAppName("SparkTFIDF")
sc = SparkContext(conf = conf)

In [4]:
# Load documents (one per line).
rawData = sc.textFile("subset-small.tsv")
fields = rawData.map(lambda x: x.split("\t"))
documents = fields.map(lambda x: x[3].split(" "))

# Store the document names for later:
documentNames = fields.map(lambda x: x[1])

#Now hash the words in each document to their term frequencies:
hashingTF = HashingTF(100000)  #100K hash buckets just to save some memory
tf = hashingTF.transform(documents)

At this point we have an RDD of sparse vectors representing each document,
where each value maps to the term frequency of each unique hash value.


In [5]:
# Let's compute the TF*IDF of each term in each document:
tf.cache()
idf = IDF(minDocFreq=2).fit(tf)
tfidf = idf.transform(tf)

Now we have an RDD of sparse vectors, where each value is the TFxIDF
of each unique hash value for each document.

I happen to know that the article for "Abraham Lincoln" is in our data
set, so let's search for "Gettysburg" (Lincoln gave a famous speech there):

First, let's figure out what hash value "Gettysburg" maps to by finding the index a sparse vector from HashingTF gives us back:

In [6]:
gettysburgTF = hashingTF.transform(["Gettysburg"])
gettysburgHashValue = int(gettysburgTF.indices[0])


Now we will extract the TF*IDF score for Gettsyburg's hash value into

In [7]:

# a new RDD for each document:
gettysburgRelevance = tfidf.map(lambda x: x[gettysburgHashValue])

# We'll zip in the document names so we can see which is which:
zippedResults = gettysburgRelevance.zip(documentNames)

In [8]:
# And, print the document with the maximum TF*IDF value:
print("Best document for Gettysburg is:")
print(zippedResults.max())


Best document for Gettysburg is:
(8.27233211415088, 'Ares')
